# SmallVessel Density Analysis

Compare SmallVessel density across tissue regions, grouped by rs3184504 (SH2B3) genotype.

**Data sources:**
- `Measurements/AnnotationsWithTT.csv` — main QuPath project (12 images incl. T/T samples, 467K rows)
- `Measurements/2008Annotations.csv` — separate QuPath project for sample HDL098 (2 images, 1.2K rows)

**Harmonization rules for 2008 data:**
- `Vessel` → `SmallVessel`, `Arteriole` → `LargeVessel`
- `Sinusoid` → `RedPulp` (splenic sinusoids are red pulp structures)
- `Red_Pulp` → `RedPulp`, `Trabecula` → `Trabeculae`
- `White_Pulp`, `Peripheral_White_Pulp` kept as-is (no direct equivalent)

**Limitation:** In the 2008 project, vessels are parented to Sinusoid (→RedPulp) or Root, NOT to Follicle/PALS. So 2008 data only contributes vessel density for RedPulp.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pathlib import Path
import re
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

PROJECT = Path('/home/smith6jt/SpleenFollicleCounterQP')
sns.set_style('whitegrid')
sns.set_context('notebook', font_scale=1.1)

## 1. Load and combine data

In [ ]:
# Common columns (both files share these first 17)
COMMON_COLS = [
    'Image', 'Object ID', 'Object type', 'Name', 'Classification',
    'Parent', 'ROI', 'Centroid X µm', 'Centroid Y µm', 'Area µm^2',
    'Perimeter µm', 'Num Detections', 'Length µm', 'Circularity',
    'Solidity', 'Max diameter µm', 'Min diameter µm'
]

# Load main data (includes T/T samples: HDL063, HDL073, 1901HBMP004)
df_main = pd.read_csv(PROJECT / 'Measurements/AnnotationsWithTT.csv', usecols=COMMON_COLS)
df_main['Source'] = 'main'
print(f'Main: {len(df_main):,} rows, {df_main["Image"].nunique()} images')

# Load 2008 data (only common columns)
df_2008 = pd.read_csv(PROJECT / 'Measurements/2008Annotations.csv', usecols=COMMON_COLS)
df_2008['Source'] = '2008'
print(f'2008: {len(df_2008):,} rows, {df_2008["Image"].nunique()} images')

In [ ]:
# Harmonize 2008 Classification names
class_map = {
    'Vessel': 'SmallVessel',
    'Arteriole': 'LargeVessel',
    'Red_Pulp': 'RedPulp',
    'Trabecula': 'Trabeculae',
    'Sinusoid': 'RedPulp',
}
df_2008['Classification'] = df_2008['Classification'].replace(class_map)

# Harmonize Parent column
parent_map = {
    'Annotation (Sinusoid)': 'Annotation (RedPulp)',
}
df_2008['Parent'] = df_2008['Parent'].replace(parent_map)

print('2008 after harmonization:')
print(df_2008['Classification'].value_counts())
print()
print(df_2008.loc[df_2008['Classification'] == 'SmallVessel', 'Parent'].value_counts())

In [ ]:
# Combine
df = pd.concat([df_main, df_2008], ignore_index=True)
print(f'Combined: {len(df):,} rows, {df["Image"].nunique()} images')
print()
print(df['Classification'].value_counts())

## 2. Map images to sample IDs and genotypes

In [ ]:
# Load genotype groups
groups = pd.read_excel(PROJECT / 'Groups.xlsx')

# Build lookup: HANDEL_ID -> genotype, ALT_ID -> HANDEL_ID
handel_geno = dict(zip(
    groups['HANDEL ID'].dropna(),
    groups.loc[groups['HANDEL ID'].notna(), 'rs3184504 (SH2B3)']
))
alt_to_handel = dict(zip(
    groups['ALT ID'].dropna().astype(int).astype(str),
    groups.loc[groups['ALT ID'].notna(), 'HANDEL ID']
))
# Also map ALT_ID directly to genotype for samples without HANDEL ID
alt_geno = dict(zip(
    groups['ALT ID'].dropna().astype(int).astype(str),
    groups.loc[groups['ALT ID'].notna(), 'rs3184504 (SH2B3)']
))

def extract_sample_id(image_name):
    """Extract sample ID from image filename."""
    # HDL011_PC33.ome.tiff -> 'HDL011'
    m = re.match(r'^(HDL\d+)', image_name)
    if m:
        return m.group(1)
    # 1901HBMP004_PC29 or 2008_CC2B -> 4-digit ALT ID
    m = re.match(r'^(\d{4})', image_name)
    if m:
        alt_id = m.group(1)
        handel = alt_to_handel.get(alt_id)
        return handel if handel and pd.notna(handel) else alt_id
    return image_name

def get_genotype(sample_id):
    """Look up genotype for a sample ID."""
    if sample_id in handel_geno:
        return handel_geno[sample_id]
    if sample_id in alt_geno:
        return alt_geno[sample_id]
    return np.nan

df['Sample'] = df['Image'].apply(extract_sample_id)
df['Genotype'] = df['Sample'].apply(get_genotype)

# Show mapping
mapping = df[['Image', 'Sample', 'Genotype', 'Source']].drop_duplicates().sort_values('Sample')
print(mapping.to_string(index=False))

In [ ]:
# Exclude samples without genotype
excluded = df.loc[df['Genotype'].isna(), 'Image'].unique()
if len(excluded):
    print(f'Excluding {len(excluded)} image(s) without genotype: {list(excluded)}')
df = df.dropna(subset=['Genotype']).copy()
print(f'After filtering: {len(df):,} rows, {df["Image"].nunique()} images, {df["Sample"].nunique()} samples')
print()
print('Samples per genotype:')
print(df.groupby('Genotype')['Sample'].nunique())

## 3. Compute vessel density per region per image

In [ ]:
# Identify region annotations and their areas
REGION_CLASSES = ['Follicle', 'PALS', 'RedPulp', 'Trabeculae', 'LargeVessel',
                  'White_Pulp', 'Peripheral_White_Pulp']

regions = df[df['Classification'].isin(REGION_CLASSES)].copy()

# Aggregate region area per image (sum if multiple annotations of same class)
region_area = (
    regions.groupby(['Image', 'Sample', 'Genotype', 'Classification'])['Area µm^2']
    .sum()
    .reset_index()
    .rename(columns={'Classification': 'Region', 'Area µm^2': 'Region_Area_um2'})
)
region_area['Region_Area_mm2'] = region_area['Region_Area_um2'] / 1e6

print('Region areas (mm²) per image:')
pivot = region_area.pivot_table(
    index='Image', columns='Region', values='Region_Area_mm2', aggfunc='sum'
)
print(pivot.round(2).to_string())

In [ ]:
# Count SmallVessels per region per image
vessels = df[df['Classification'] == 'SmallVessel'].copy()

# Extract parent region from Parent column: 'Annotation (Follicle)' -> 'Follicle'
vessels['Region'] = vessels['Parent'].str.extract(r'Annotation \((.+)\)')

# Drop vessels not assigned to a region
unassigned = vessels['Region'].isna().sum()
print(f'Vessels not assigned to a region (Parent=Root): {unassigned:,}')
vessels = vessels.dropna(subset=['Region'])

vessel_counts = (
    vessels.groupby(['Image', 'Sample', 'Genotype', 'Region'])
    .size()
    .reset_index(name='Vessel_Count')
)

print(f'\nVessel counts per region:')
print(vessel_counts.groupby('Region')['Vessel_Count'].sum())

In [ ]:
# Merge vessel counts with region areas -> density
density = vessel_counts.merge(
    region_area[['Image', 'Region', 'Region_Area_um2', 'Region_Area_mm2']],
    on=['Image', 'Region'],
    how='left'
)
density['Density_per_mm2'] = density['Vessel_Count'] / density['Region_Area_mm2']

# Add RedPulp density per image for normalization
rp_density = (
    density[density['Region'] == 'RedPulp']
    [['Image', 'Density_per_mm2']]
    .rename(columns={'Density_per_mm2': 'RedPulp_Density'})
)
density = density.merge(rp_density, on='Image', how='left')
density['Density_Normalized'] = density['Density_per_mm2'] / density['RedPulp_Density']

print(density[['Image', 'Region', 'Vessel_Count', 'Region_Area_mm2', 
               'Density_per_mm2', 'Density_Normalized', 'Genotype']]
      .sort_values(['Image', 'Region']).to_string(index=False))

## 4. Summary statistics

In [ ]:
# Summary by region and genotype
summary = (
    density.groupby(['Region', 'Genotype'])['Density_per_mm2']
    .agg(['count', 'mean', 'std', 'median'])
    .round(1)
    .reset_index()
)
summary.columns = ['Region', 'Genotype', 'N', 'Mean', 'SD', 'Median']
print('Vessel density (per mm²) by region and genotype:')
print(summary.to_string(index=False))

## 5. Visualization

In [ ]:
# Focus on main tissue regions
MAIN_REGIONS = ['Follicle', 'PALS', 'RedPulp', 'Trabeculae']
GENO_ORDER = ['C/C', 'C/T', 'T/T']
plot_data = density[density['Region'].isin(MAIN_REGIONS)].copy()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Raw density
sns.boxplot(data=plot_data, x='Region', y='Density_per_mm2', hue='Genotype',
            order=MAIN_REGIONS, hue_order=GENO_ORDER, palette='Set2', ax=axes[0])
sns.stripplot(data=plot_data, x='Region', y='Density_per_mm2', hue='Genotype',
              order=MAIN_REGIONS, hue_order=GENO_ORDER, palette='Set2', dodge=True, alpha=0.7,
              size=6, ax=axes[0], legend=False)
axes[0].set_title('SmallVessel Density by Region & Genotype')
axes[0].set_ylabel('Vessels per mm²')
axes[0].set_xlabel('')

# Normalized to RedPulp
sns.boxplot(data=plot_data, x='Region', y='Density_Normalized', hue='Genotype',
            order=MAIN_REGIONS, hue_order=GENO_ORDER, palette='Set2', ax=axes[1])
sns.stripplot(data=plot_data, x='Region', y='Density_Normalized', hue='Genotype',
              order=MAIN_REGIONS, hue_order=GENO_ORDER, palette='Set2', dodge=True, alpha=0.7,
              size=6, ax=axes[1], legend=False)
axes[1].set_title('Vessel Density Normalized to RedPulp')
axes[1].set_ylabel('Ratio (region / RedPulp)')
axes[1].set_xlabel('')
axes[1].axhline(y=1.0, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig(PROJECT / 'analysis/vessel_density_by_genotype.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Per-sample density heatmap
heat_data = density[density['Region'].isin(MAIN_REGIONS)].pivot_table(
    index='Sample', columns='Region', values='Density_per_mm2'
)[MAIN_REGIONS]

# Sort by genotype then sample name
geno_map = density.drop_duplicates('Sample').set_index('Sample')['Genotype']
sort_keys = pd.DataFrame({'Sample': heat_data.index, 'Genotype': [geno_map[s] for s in heat_data.index]})
sort_keys = sort_keys.sort_values(['Genotype', 'Sample'])
heat_data = heat_data.loc[sort_keys['Sample']]

fig, ax = plt.subplots(figsize=(8, 7))
sns.heatmap(heat_data, annot=True, fmt='.0f', cmap='YlOrRd', ax=ax,
            linewidths=0.5, cbar_kws={'label': 'Vessels per mm²'})
labels = [f'{s} ({geno_map[s]})' for s in heat_data.index]
ax.set_yticklabels(labels, rotation=0)
ax.set_title('SmallVessel Density (per mm²) — Samples × Regions')
plt.tight_layout()
plt.savefig(PROJECT / 'analysis/vessel_density_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Statistical tests — Follicle focus

In [ ]:
# Test vessel density differences between genotypes for each region
results = []
for region in MAIN_REGIONS:
    rd = density[density['Region'] == region]
    genotypes = rd['Genotype'].unique()
    if len(genotypes) < 2:
        continue
    groups_data = [rd.loc[rd['Genotype'] == g, 'Density_per_mm2'].values for g in sorted(genotypes)]
    
    # Kruskal-Wallis if 3 groups, Mann-Whitney if 2
    if len(groups_data) == 2:
        stat, p = stats.mannwhitneyu(groups_data[0], groups_data[1], alternative='two-sided')
        test_name = 'Mann-Whitney U'
    else:
        stat, p = stats.kruskal(*groups_data)
        test_name = 'Kruskal-Wallis'
    
    results.append({
        'Region': region,
        'Test': test_name,
        'Statistic': round(stat, 2),
        'p-value': p,
        'N per group': ', '.join(f'{g}={len(d)}' for g, d in zip(sorted(genotypes), groups_data)),
    })

results_df = pd.DataFrame(results)
results_df['p-value'] = results_df['p-value'].apply(lambda x: f'{x:.4f}' if x >= 0.001 else f'{x:.2e}')
print('Genotype comparison — vessel density per region:')
print(results_df.to_string(index=False))

In [ ]:
# Follicle-specific analysis with pairwise comparisons
foll = density[density['Region'] == 'Follicle'].copy()
print('=== FOLLICLE VESSEL DENSITY ===')
print()
for geno in sorted(foll['Genotype'].unique()):
    vals = foll.loc[foll['Genotype'] == geno, 'Density_per_mm2']
    print(f'{geno}: n={len(vals)}, mean={vals.mean():.1f}, '
          f'median={vals.median():.1f}, SD={vals.std():.1f} vessels/mm²')

print()
# Pairwise if 3 genotypes
geno_list = sorted(foll['Genotype'].unique())
if len(geno_list) >= 2:
    from itertools import combinations
    print('Pairwise Mann-Whitney U tests:')
    for g1, g2 in combinations(geno_list, 2):
        d1 = foll.loc[foll['Genotype'] == g1, 'Density_per_mm2'].values
        d2 = foll.loc[foll['Genotype'] == g2, 'Density_per_mm2'].values
        if len(d1) > 0 and len(d2) > 0:
            stat, p = stats.mannwhitneyu(d1, d2, alternative='two-sided')
            print(f'  {g1} vs {g2}: U={stat:.1f}, p={p:.4f}')

In [ ]:
# Follicle density strip+box by genotype
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.boxplot(data=foll, x='Genotype', y='Density_per_mm2', palette='Set2',
            order=GENO_ORDER, ax=axes[0])
sns.stripplot(data=foll, x='Genotype', y='Density_per_mm2', palette='Set2',
              order=GENO_ORDER, size=8, alpha=0.7, ax=axes[0])
axes[0].set_title('Follicle Vessel Density by Genotype')
axes[0].set_ylabel('Vessels per mm²')

sns.boxplot(data=foll, x='Genotype', y='Density_Normalized', palette='Set2',
            order=GENO_ORDER, ax=axes[1])
sns.stripplot(data=foll, x='Genotype', y='Density_Normalized', palette='Set2',
              order=GENO_ORDER, size=8, alpha=0.7, ax=axes[1])
axes[1].set_title('Follicle Vessel Density (Normalized to RedPulp)')
axes[1].set_ylabel('Ratio (Follicle / RedPulp)')
axes[1].axhline(y=1.0, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig(PROJECT / 'analysis/follicle_density_by_genotype.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Export results

In [ ]:
# Save density table
density.to_csv(PROJECT / 'analysis/vessel_density_results.csv', index=False)
print(f'Saved: analysis/vessel_density_results.csv ({len(density)} rows)')

# Save summary
summary.to_csv(PROJECT / 'analysis/vessel_density_summary.csv', index=False)
print(f'Saved: analysis/vessel_density_summary.csv')